In [9]:
def searchRedditThreads(subreddit, search_term):
    import numpy as np
    import pandas as pd

    import requests, json, time, datetime
    import praw
    
    # login credentials
    r_creds = {
        "id" : "",
        "secret" : "",
        "user" : "Adventurous_Party788",
        "pass" : ""
    }
    
    # read in credentials
    creds_file = open('creds.json', "r")
    r_creds = json.loads(creds_file.read())
    
    reddit = praw.Reddit(
        client_id = r_creds['id'],
        client_secret = r_creds['secret'],
        username = r_creds['user'],
        password = r_creds['pass'],
        user_agent = 'michael'
    )
    
    # access subreddit
    subredditObj = reddit.subreddit(subreddit)
    
    categories = ['hot','new','top','rising']
    dfs = []
    
    # loop through 4 categories (hot, new, top, rising)
    for category in categories:
        if category == 'hot':
            threads = list(subredditObj.hot(limit=10))
        elif category == 'new':
            threads = list(subredditObj.new(limit=10))
        elif category == 'top':
            threads = list(subredditObj.top(limit=10))
        else:
            threads = list(subredditObj.rising(limit=10))
        
        title = [post.title for post in threads]
        body_text = [post.selftext for post in threads]
        body_word_count = [len(post.selftext.split()) for post in threads]
        comment_bodies = []
        
        # get all comments for each of the threads
        for thread in threads:
            thread.comments.replace_more(limit=None)
            allComments = ''
            for comment in thread.comments.list():
                allComments += comment.body + '\n'
            comment_bodies.append(allComments.strip())
        
        comments_word_count = [len(comment.split()) for comment in comment_bodies]
        up_votes = [post.ups for post in threads]
        url = [post.url for post in threads]
        
        thread_dict = {
            'title': title,
            'body_text' : body_text,
            'body_word_count' : body_word_count,
            'comment_bodies' : comment_bodies,
            'comments_word_count' : comments_word_count,
            'up_votes' : up_votes,
            'url' : url,
            'category' : category,
            'subreddit' : subreddit
        }
        
        # create and add dataframe to dfs list
        dfs.append(pd.DataFrame(thread_dict))    
    
    # combine dfs
    df = dfs[0].append(dfs[1:])

    # add 'all_text' column w/ title, post body, comments
    all_text = []
    for index, row in df.iterrows():
        all_text.append(row['title'] + '\n' + row['body_text'] + row['comment_bodies'])
    df['all_text'] = all_text
    
    # count num occurences of search term in all_text column (occurrence = found at least once in the row)
    numOccurences = 0
    for index, row in df.iterrows():
        if search_term in row['all_text'].lower():
            numOccurences += 1
    print(numOccurences)
    
    df.to_csv('./data/' + subreddit + '_thread_posts_comments.csv', encoding='utf-8', index=False)

In [10]:
searchRedditThreads('remnantgame', 'dark souls')

5


/var/folders/8t/9lr88cgn3xb54w91878m0c1m0000gn/T/ipykernel_36390/1937240981.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = dfs[0].append(dfs[1:])
